In [19]:
import torch
print(torch.version)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

<module 'torch.version' from '/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/version.py'>
True
4
0
NVIDIA A10G


In [20]:
!git clone https://github.com/arthurcornelio88/stable-audio-tools-sam.git

fatal: destination path 'stable-audio-tools-sam' already exists and is not an empty directory.


In [21]:
!pip install stable-audio-tools

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [22]:
import os
os.environ["WANDB_API_KEY"] = "5fba2404cdb24b50df5ac10cf95fc5a9099f7b2d"

In [23]:
!wandb login

wandb: Currently logged in as: rayan-anser (sammusic). Use `wandb login --relogin` to force relogin


In [24]:
!conda install -y -c conda-forge libsndfile

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/pytorch_p310

  added / updated specs:
    - libsndfile


The following packages will be UPDATED:

  certifi                             2024.2.2-pyhd8ed1ab_0 --> 2024.7.4-pyhd8ed1ab_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [25]:
!pip install soundfile

In [26]:
pip install torch torchvision


Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install flash-attn

Note: you may need to restart the kernel to use updated packages.


In [28]:
import boto3

s3 = boto3.resource('s3')
bucket_name = 'sagemaker-eu-west-2-339470605340'
tar_file_key = 'webdataset_audio/audio_dataset-00001.tar'  # The key (path) to your tar archive in S3

try:
    obj = s3.Object(bucket_name, tar_file_key).get()
    print("Successfully accessed tar archive!")
except Exception as e:
    print(f"Error accessing tar archive: {e}")

Successfully accessed tar archive!


In [29]:
#télécharge le model en local
import boto3

s3 = boto3.client('s3')
s3.download_file('sagemaker-eu-west-2-339470605340', 'model.safetensors', 'model.safetensors')
#puis mettre le fichier dans stable_open_model_files

In [30]:
#installe les bonnes version pour torch et torch audio
!pip install torch torchaudio

In [31]:
!nvidia-smi


Sat Jul 13 13:58:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1B.0 Off |                    0 |
|  0%   30C    P0              58W / 300W |    256MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [32]:
import torch
torch.cuda.set_per_process_memory_fraction(0.9)

In [33]:
import torch
torch.backends.cuda.max_split_size_mb = 128

In [34]:
pip install -U deepspeed

Note: you may need to restart the kernel to use updated packages.


In [35]:
#lance l'entrainement
!python stable-audio-tools-sam/train.py --model-config stable_open_model_files/model_config.json --dataset-config stable_open_model_files/dataset_config.json --name rayan-training --save-dir checkpoints --pretrained-ckpt-path stable_open_model_files/model.safetensors --batch-size 16 --num-gpus 4 --strategy deepspeed


Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/train.py", line 176, in <module>
    main()
  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/train.py", line 102, in main
    model = create_model_from_config(model_config)
  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/stable_audio_tools/models/factory.py", line 15, in create_model_from_config
    from .diffusion import create_diffusion_cond_from_config
  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/stable_audio_tools/models/diffusion.py", line 13, in <module>
    from ..inference.generation import generate_diffusion_cond
  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/stable_audio_tools/inference/generation.py", line 8, in <module>
    from .sampling import sample, sample_k, sample_rf
  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/stable_audio_tools/inference/sampling.py", line 5, in <module>
    import k_diffusion as K
  File "/home/ec2-user/anaconda3/

In [6]:
!python stable-audio-tools-sam/train.py -h

usage: train.py [-h] --model-config MODEL_CONFIG --dataset-config
                DATASET_CONFIG [--name NAME] [--save-dir SAVE_DIR]
                [--pretrained-ckpt-path PRETRAINED_CKPT_PATH] [--seed SEED]
                [--num-workers NUM_WORKERS]
                [--checkpoint-every CHECKPOINT_EVERY] [--strategy STRATEGY]
                [--num-gpus NUM_GPUS] [--num-nodes NUM_NODES]
                [--precision {32,16,bf16,16-mixed}]
                [--accum-batches ACCUM_BATCHES]
                [--gradient-clip-val GRADIENT_CLIP_VAL]
                [--remove-pretransform-weight-norm {None,pre_load,post_load}]
                [--pretransform-ckpt-path PRETRANSFORM_CKPT_PATH]
                [--batch-size BATCH_SIZE] [--ckpt-path CKPT_PATH]

Training script for Stable Audio Tools

options:
  -h, --help            show this help message and exit
  --model-config MODEL_CONFIG
                        Path to model config JSON file (relative to
                        project_base_pa